In [25]:
# Helper function for visualization.
%matplotlib inline

import torch.optim as optim

from torch_geometric.data import DataLoader
import torch


**Graph Neural Networks (GNNs) to the task of node classification**.
Here, we are given the ground-truth labels of only a small subset of nodes, and want to infer the labels for all the remaining nodes (*transductive learning*).

To demonstrate, we make use of the `Cora` dataset, which is a **citation network** where nodes represent documents.
Each node is described by a 1433-dimensional bag-of-words feature vector.
Two documents are connected if there exists a citation link between them.
The task is to infer the category of each document (7 in total).


In [26]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


In [27]:
torch.set_printoptions(edgeitems=300)

In [28]:
data.x

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [29]:
data.num_nodes

2708

In [30]:
torch.count_nonzero(data.train_mask)

tensor(140)

In [31]:
data.train_mask.sum().item()

140

In [32]:
data.val_mask.sum().item()

500

In [33]:
data.test_mask.sum().item()

1000

In [34]:
data.y.sum()

tensor(7781)

In [35]:
from src.GCN import GCNStack
model = GCNStack(data.num_node_features, hidden_dim1=128, hidden_dim2=32, hidden_dim3=16, output_dim=dataset.num_classes)
print(model)

GCNStack(
  (convs): ModuleList(
    (0): GCNConv(1433, 128)
    (1): GCNConv(128, 32)
    (2): GCNConv(32, 16)
  )
  (lns): ModuleList(
    (0): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  )
  (post_mp): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [36]:
def model_test(loader, model, is_validation=False, is_training=False):
    ''' Testing Code of the Model '''
    model.eval()

    correct = 0
    for data in loader:
        with torch.no_grad():
            emb, pred = model(data.x, data.edge_index)
            pred = pred.argmax(dim=1)
            label = data.y

        if is_training:
            mask = data.val_mask if is_validation else data.train_mask
        else: # testing
            mask = data.val_mask if is_validation else data.test_mask
        # node classification: only evaluate on nodes in test set
        pred = pred[mask]
        label = data.y[mask]

        correct += pred.eq(label).sum().item()
    total = 0
    for data in loader.dataset:
        if is_training:
            total += torch.sum(data.train_mask).item()
        else:
            total += torch.sum(data.test_mask).item()
    return correct / total

def model_train(dataset, writer, model, epoch_num, lr, weight_decay):
    ''' Training code of the model '''
    test_loader = loader = DataLoader(dataset, shuffle=False)

    # Optimizer
    # opt = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=0.9)
    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # visualize the model architecture in tensorboard
    # writer.add_graph(model, ( data.x, data.edge_index ))

    # Training:
    for epoch in range(epoch_num + 1):
        total_loss = 0
        model.train()
        for batch in loader:
            #print(batch.train_mask, '----')
            opt.zero_grad()
            embedding, pred = model(batch.x, batch.edge_index)
            label = batch.y
            pred = pred[batch.train_mask]
            label = label[batch.train_mask]
            loss = model.loss(pred, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        writer.add_scalar("loss", total_loss, epoch)

        if epoch % 10 == 0:
            train_acc = model_test(test_loader, model, is_training=True)
            test_acc = model_test(test_loader, model, is_training=False)
            print("Epoch {}. Loss: {:.4f}. Train accuracy: {:.4f}. Test accuracy: {:.4f}".format(
                epoch, total_loss, train_acc, test_acc))
            writer.add_scalar("test accuracy", test_acc, epoch)

        if epoch % 20 == 0:
            name = 'epoch' + str(epoch)
            writer.add_embedding(embedding, global_step=epoch, tag=name, metadata=batch.y)

    return model

from datetime import datetime
from tensorboardX import SummaryWriter

writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

model = model_train(dataset, writer, model, epoch_num=200, lr=0.01, weight_decay=5e-4)

Epoch 0. Loss: 1.9679. Train accuracy: 0.1500. Test accuracy: 0.3230
Epoch 10. Loss: 1.2661. Train accuracy: 0.7714. Test accuracy: 0.5250
Epoch 20. Loss: 0.6400. Train accuracy: 0.9643. Test accuracy: 0.7100
Epoch 30. Loss: 0.5002. Train accuracy: 0.9929. Test accuracy: 0.7310
Epoch 40. Loss: 0.3952. Train accuracy: 0.9929. Test accuracy: 0.7420
Epoch 50. Loss: 0.3544. Train accuracy: 1.0000. Test accuracy: 0.7710
Epoch 60. Loss: 0.4583. Train accuracy: 1.0000. Test accuracy: 0.7540
Epoch 70. Loss: 0.3356. Train accuracy: 0.9929. Test accuracy: 0.6960
Epoch 80. Loss: 0.3128. Train accuracy: 1.0000. Test accuracy: 0.6910
Epoch 90. Loss: 0.2700. Train accuracy: 1.0000. Test accuracy: 0.7600
Epoch 100. Loss: 0.2647. Train accuracy: 1.0000. Test accuracy: 0.7680
Epoch 110. Loss: 0.2342. Train accuracy: 1.0000. Test accuracy: 0.7360
Epoch 120. Loss: 0.3185. Train accuracy: 1.0000. Test accuracy: 0.7900
Epoch 130. Loss: 0.3387. Train accuracy: 1.0000. Test accuracy: 0.7580
Epoch 140. Loss: 